In [1]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import os, json
from pyspark.sql.functions import *

spark = (
    SparkSession.builder.appName("CNN")
    .getOrCreate()
)

In [2]:
path_to_json = '../datalake/bronze/cnn/'
json_files = [  '../datalake/bronze/cnn/' + pos_json for pos_json in os.listdir(path_to_json) 
              if pos_json.endswith('.json')]

df = (
    spark
    .read
    .format("json")
    .load(json_files, multiLine=True)
)

In [3]:
cols = ["author", "messageEx"]
df = (
    df
    .withColumn("badgeUrl", col("author.badgeUrl"))
    .withColumn("channelId", col("author.channelId"))
    .withColumn("channelUrl", col("author.channelUrl"))
    .withColumn("imageUrl", col("author.imageUrl"))
    .withColumn("isChatModerator", col("author.isChatModerator"))
    .withColumn("isChatOwner", col("author.isChatOwner"))
    .withColumn("isChatSponsor", col("author.isChatSponsor"))
    .withColumn("isVerified", col("author.isVerified"))
    .withColumn("name", col("author.name"))
    .withColumn("type", col("author.type"))
    .withColumn("year", year(col("datetime")))
    .withColumn("month", month(col("datetime")))
    .drop(*cols)

)

In [4]:
df_pandas = df.toPandas()

In [5]:
df_pandas.to_parquet(
    '../datalake/silver/cnn/data.parquet',
    partition_cols=['year', 'month'])